针对问题一本次赛题为长序列时间序列预测任务，对数据进行了处理与特征提取。代码首先对数据进行可视化，其次对数据进行特征工程，最后通过机器学习catboost、xgboost、lightgbm进行预测，包括数据清洗、特征工程、模型预测以及后期优化策略，神经网络代码LSTM以及时序prophet模型、ARIMA等。

针对问题二，时间突变检测，包括MK突变检验，统计学检验，t检验，多种检验方式。

In [ ]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.io as pio
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from chart_studio import plotly as py
import matplotlib.pyplot as plt
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore')
from pylab import mpl 
mpl.rcParams['font.sans-serif'] = ['FangSong'] # 指定默认字体 
mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib  inline
plt.style.use('seaborn-darkgrid')
sns.set(style = 'darkgrid')
import warnings
warnings.filterwarnings("ignore")
import lightgbm as lgb  
from sklearn.preprocessing import scale
from sklearn import metrics
import xgboost as xgb 
from catboost import CatBoostRegressor
import time
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import matplotlib
from colorama import Fore

from sklearn.metrics import mean_absolute_error, mean_squared_error
import math
import warnings 
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

## 读取数据集

In [ ]:
data1 = pd.read_csv(r'附件1-区域15分钟负荷数据.csv')
data2 = pd.read_csv(r'附件2-行业日负荷数据.csv')
data3 = pd.read_csv(r'附件3-气象数据.csv')

In [ ]:
data3.describe()

### 附件三数据预处理

In [ ]:
data3 = data3[1:]
#温度特征处理
data3['最高温度'] = data3['最高温度'].map(lambda d: d.replace('℃','')).astype(int)
data3['最低温度'] = data3['最低温度'].map(lambda d: d.replace('℃','')).astype(int)
#天气状况特征处理
series = data3.join(data3['天气状况'].str.split('/',expand=True))
data3['天气1'] = series[0]
data3['天气2'] = series[1]
#风向特征处理
dic = {'无持续风向<3级':0, '北风4～5级':3, '微风<3级':1, '北风3':2, '东北风3～4级':2, '北风3～4级':2,
       '南风3～4级':2, '南风4～5级':3, '东北偏东风2':1, '无持续风向微风':0, '无持续风向1-2级':1, '东风3-4级':2,
       '东南风4-5级':3, '东风3～4级':2, '东北风3-4级':2, '东风8-9级':4, '东南风3-4级':2, '北风3-4级':2,
       '北风4-5级':3, '南风3-4级':2, '南风1-2级':1, '东南风1-2级':1, '西南风3-4级':2, '东风1-2级':1,
       '北风1-2级':1, '东北风1-2级':1, '西南风1-2级':1,'东风3-4级':2,'东南风3～4级':2,'北风3-4级':2,'北风4-5级':3}
data3['白天风力风向'] = data3['白天风力风向'].map(dic)
data3['夜晚风力风向'] = data3['夜晚风力风向'].map(dic)
#天气特征进行有序编码
dic1 = {'晴':1, '多云':2, '阴':3,  '小雨':4, '小雨-中雨':5,  '中雨':6, '中雨-大雨':7, '大雨':8,'阵雨':9,'雷阵雨':10}
data3['天气1'] = data3['天气1'].map(dic1)
data3['天气2'] = data3['天气2'].map(dic1)
# del data3['天气状况']
data3 = data3.fillna(method = 'ffill') 

In [ ]:
xtick = data3.日期.map(lambda x:x[7:-1])[365:31+365]
plt.figure(figsize=(15,4))
plt.plot(data3.日期.values[365:31+365],data3.白天风力风向.values[365:31+365])
plt.plot(data3.日期.values[365:31+365],data3.夜晚风力风向.values[365:31+365])
plt.title('2019年一月白天/夜晚风力风向')
plt.xticks(np.arange(len(xtick)), xtick, )
plt.xlabel('日期(1月)')
plt.ylabel('风力风向')
plt.legend(['白天风力风向','夜晚风力风向'])
plt.savefig('2019年一月风力风向折线图.png')
plt.show()

In [ ]:
xtick = data3.日期.map(lambda x:x[7:-1])[:31]
plt.figure(figsize=(15,4))
plt.plot(data3.日期.values[:31],data3.白天风力风向.values[:31])
plt.plot(data3.日期.values[:31],data3.夜晚风力风向.values[:31])
plt.title('2018年一月白天/夜晚风力风向')
plt.xticks(np.arange(len(xtick)), xtick, )
plt.xlabel('日期(1月)')
plt.ylabel('风力风向')
plt.legend(['白天风力风向','夜晚风力风向'])
plt.savefig('2018年一月风力风向折线图.png')
plt.show()

In [ ]:
xtick = data3.日期.map(lambda x:x[7:-1])[365:31+365]
plt.figure(figsize=(15,4))
plt.plot(data3.日期.values[365:31+365],data3.天气1.values[365:31+365])
plt.plot(data3.日期.values[365:31+365],data3.天气2.values[365:31+365])
plt.title('2019年一月天气状况')
plt.xticks(np.arange(len(xtick)), xtick, )
plt.xlabel('日期(1月)')
plt.ylabel('天气状况')
# plt.legend(['初始天气状况','天气状况'])
plt.savefig('2019年一月天气状况折线图.png')
plt.show()

In [ ]:
xtick = data3.日期.map(lambda x:x[7:-1])[:31]
plt.figure(figsize=(15,4))
plt.plot(data3.日期.values[:31],data3.天气1.values[:31])
plt.plot(data3.日期.values[:31],data3.天气2.values[:31])
plt.title('2018年一月天气状况')
plt.xticks(np.arange(len(xtick)), xtick, )
plt.xlabel('日期(1月)')
plt.ylabel('天气状况')
# plt.legend(['白天风力风向','夜晚风力风向'])
plt.savefig('2018年一月天气状况折线图.png')
plt.show()

In [ ]:
xtick = data3.日期.map(lambda x:x[7:-1])[365:31+365]
plt.figure(figsize=(15,4))
plt.plot(data3.日期.values[365:31+365],data3.最高温度.values[365:31+365])
plt.plot(data3.日期.values[365:31+365],data3.最低温度.values[365:31+365])
plt.title('2019年一月最高/最低温度')
plt.xticks(np.arange(len(xtick)), xtick, )
plt.xlabel('日期(1月)')
plt.ylabel('温度')
plt.legend(['最高温度','最低温度'])
plt.savefig('2019年一月温度折线图.png')
plt.show()

In [ ]:
xtick = data3.日期.map(lambda x:x[7:-1])[:31]
plt.figure(figsize=(15,4))
plt.plot(data3.日期.values[:31],data3.最高温度.values[:31])
plt.plot(data3.日期.values[:31],data3.最低温度.values[:31])
plt.title('2018年一月最高/最低温度')
plt.xticks(np.arange(len(xtick)), xtick, )
plt.xlabel('日期(1月)')
plt.ylabel('温度')
plt.legend(['最高温度','最低温度'])
plt.savefig('2018年一月温度折线图.png')
plt.show()

In [ ]:
#合并附件1，3
data1 = data1.rename(columns={'数据时间':'日期1'})
data1['日期'] = data1['日期1'].apply(lambda x: x.split(' ')[0])
data3.loc[:, '日期'] = pd.to_datetime(data3.loc[:, '日期'], format='%Y年%m月%d日', errors='coerce')
data1.loc[:, '日期'] = pd.to_datetime(data1.loc[:, '日期'], format='%Y/%m/%d', errors='coerce')
train = data1.merge(data3,on='日期',how='left')
# del train['日期']
train  = train.rename(columns={'日期1':'date'})
del train['日期']
del train['天气状况']
train.to_csv('当地天气状况处理.csv')

## 第一问

In [ ]:
df = pd.read_csv(r"当地天气状况处理.csv")
df = df.drop(['Unnamed: 0'], axis=1)

In [ ]:
df

In [ ]:
from datetime import datetime, date 

df['date'] = pd.to_datetime(df['date'])
df.head().style.set_properties(subset=['date'], **{'background-color': 'yellow'})

In [ ]:
df1 = df[:24*4*15]

f, ax = plt.subplots(nrows=7, ncols=1, figsize=(15, 30))


for i, column in enumerate(df1.drop('date', axis=1).columns):
    sns.lineplot(x=df1['date'], y=df1[column].fillna(method='ffill'), ax=ax[i], color='dodgerblue')
#     ax[i].set_title('Feature: {}'.format(column), fontsize=14)
    ax[i].set_ylabel(ylabel=column, fontsize=14)
                      
    ax[i].set_xlim([date(2018, 1, 1), date(2018, 1, 15)])   

In [ ]:
#缺失值、异常值处理
f, ax = plt.subplots(nrows=2, ncols=1, figsize=(15, 15))

df1['总有功功率（kw）'] = df1['总有功功率（kw）'].replace(np.nan, 180000)
ol = df1[df1['总有功功率（kw）']==180000]['总有功功率（kw）'].copy()
sns.lineplot(x=df1['date'], y=df1['总有功功率（kw）'].fillna(np.inf), ax=ax[0], color='dodgerblue', label='modified')
sns.lineplot(x=df1['date'], y=ol, ax=ax[0], color='red', label='original')

ax[0].set_title('Feature: 总有功功率（kw）', fontsize=14)
ax[0].set_ylabel(ylabel='总有功功率（kw）', fontsize=14)
ax[0].set_xlim([date(2018, 1, 1), date(2018, 1, 15)])

o = df1['最高温度'].copy()
df1['最高温度'] = df1['最高温度'].replace(0, np.nan)

sns.lineplot(x=df1['date'], y=o, ax=ax[1], color='darkorange', label='original')
sns.lineplot(x=df1['date'], y=df1['最高温度'].fillna(np.inf), ax=ax[1], color='dodgerblue', label='modified')
ax[1].set_title('Feature: 最高温度', fontsize=14)
ax[1].set_ylabel(ylabel='最高温度', fontsize=14)
ax[1].set_xlim([date(2018, 1, 1), date(2018, 1, 15)])

In [ ]:
f, ax = plt.subplots(nrows=4, ncols=1, figsize=(15, 12))

sns.lineplot(x=df['date'], y=df['总有功功率（kw）'].fillna(0), ax=ax[0], color='darkorange', label = 'modified')
sns.lineplot(x=df['date'], y=df['总有功功率（kw）'].fillna(np.inf), ax=ax[0], color='dodgerblue', label = 'original')
ax[0].set_title('Fill NaN with 0', fontsize=14)
ax[0].set_ylabel(ylabel='Volume', fontsize=14)

mean_drainage = df['总有功功率（kw）'].mean()
sns.lineplot(x=df['date'], y=df['总有功功率（kw）'].fillna(mean_drainage), ax=ax[1], color='darkorange', label = 'modified')
sns.lineplot(x=df['date'], y=df['总有功功率（kw）'].fillna(np.inf), ax=ax[1], color='dodgerblue', label = 'original')
ax[1].set_title(f'Fill NaN with Mean Value ({mean_drainage:.0f})', fontsize=14)
ax[1].set_ylabel(ylabel='Volume', fontsize=14)

sns.lineplot(x=df['date'], y=df['总有功功率（kw）'].ffill(), ax=ax[2], color='darkorange', label = 'modified')
sns.lineplot(x=df['date'], y=df['总有功功率（kw）'].fillna(np.inf), ax=ax[2], color='dodgerblue', label = 'original')
ax[2].set_title(f'FFill', fontsize=14)
ax[2].set_ylabel(ylabel='Volume', fontsize=14)

sns.lineplot(x=df['date'], y=df['总有功功率（kw）'].interpolate(), ax=ax[3], color='darkorange', label = 'modified')
sns.lineplot(x=df['date'], y=df['总有功功率（kw）'].fillna(np.inf), ax=ax[3], color='dodgerblue', label = 'original')
ax[3].set_title(f'Interpolate', fontsize=14)
ax[3].set_ylabel(ylabel='Volume', fontsize=14)

for i in range(4):
    ax[i].set_xlim([date(2018, 1, 1), date(2018, 1, 15)])
    
plt.tight_layout()
plt.show()

In [ ]:
df['总有功功率（kw）'] = df['总有功功率（kw）'].interpolate()

In [ ]:
#平稳性检验
rolling_window = 52
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 6))

sns.lineplot(x=df1['date'], y=df1['总有功功率（kw）'], color='dodgerblue')
sns.lineplot(x=df1['date'], y=df1['总有功功率（kw）'].rolling(rolling_window).mean(), color='black', label='rolling mean')
sns.lineplot(x=df1['date'], y=df1['总有功功率（kw）'].rolling(rolling_window).std(), color='orange', label='rolling std')
ax.set_title('Depth to Groundwater: Non-stationary \nnon-constant mean & non-constant variance', fontsize=14)
ax.set_ylabel(ylabel='总有功功率（kw） Volume', fontsize=14)
ax.set_xlim([date(2018, 1, 1), date(2018, 1, 15)])
plt.tight_layout()
plt.savefig('20180101_0115平稳性.png')
plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(df['总有功功率（kw）'].values)
result

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 6))

def visualize_adfuller_results(series, title, ax=ax):
    result = adfuller(series)
    significance_level = 0.05
    adf_stat = result[0]
    p_val = result[1]
    crit_val_1 = result[4]['1%']
    crit_val_5 = result[4]['5%']
    crit_val_10 = result[4]['10%']

    if (p_val < significance_level) & ((adf_stat < crit_val_1)):
        linecolor = 'red' 
    elif (p_val < significance_level) & (adf_stat < crit_val_5):
        linecolor = 'orange'
    elif (p_val < significance_level) & (adf_stat < crit_val_10):
        linecolor = 'forestgreen'
    else:
        linecolor = 'purple'
    sns.lineplot(x=df1['date'], y=series, ax=ax, color=linecolor)
    ax.set_title(f'ADF Statistic {adf_stat:0.3f}, p-value: {p_val:0.3f}\nCritical Values 1%: {crit_val_1:0.3f}, 5%: {crit_val_5:0.3f}, 10%: {crit_val_10:0.3f}', fontsize=14)
    ax.set_ylabel(ylabel=title, fontsize=14)

# visualize_adfuller_results(df1['最高温度'].values, 'Temperature', ax[1, 0])
visualize_adfuller_results(df1['总有功功率（kw）'].values, '总有功功率（kw）', )

plt.tight_layout()
plt.savefig('总有功功率（kw）平稳性.png')
plt.show()

In [ ]:
#时序提取
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df['day'] = pd.DatetimeIndex(df['date']).day
df['day_of_year'] = pd.DatetimeIndex(df['date']).dayofyear
df['week_of_year'] = pd.DatetimeIndex(df['date']).weekofyear
df['quarter'] = pd.DatetimeIndex(df['date']).quarter
df['season'] = df['month'] % 12 // 3 + 1

df[['date', 'year', 'month', 'day', 'day_of_year', 'week_of_year', 'quarter', 'season']].head()

## 模型评价

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred - y_true) / y_true))

y_true = np.array(y_val)  #真实值
y_pred = np.array(pred_val_y)  #预测值

print('MSE:',metrics.mean_squared_error(y_true, y_pred))

print('RMSE:',np.sqrt(metrics.mean_squared_error(y_true, y_pred)))

print('MAE:',metrics.mean_absolute_error(y_true, y_pred))

print('MAPE:',mape(y_true, y_pred))

## R2-score
from sklearn.metrics import r2_score
print('R2-score:',r2_score(y_true, y_pred))

# 第二问第一题

In [ ]:
def plot_anomaly(ts,anomaly_pred = None,anomaly_true=None,file_name = 'file'):
    fig = go.Figure()
    yhat = go.Scatter(
    x = ts.index,
    y = ts,
    mode = 'lines',  name = ts.name)
    fig.add_trace(yhat)
    if  anomaly_pred is not None:
        status = go.Scatter(
        x = anomaly_pred.index,
        y = ts.loc[anomaly_pred.index],
        mode = 'markers',  name = anomaly_pred.name,marker= {'color':'red','size':10,'symbol':'star','line_width':0})
        fig.add_trace(status)
    if  anomaly_true is not None:
        status = go.Scatter(
        x = anomaly_true.index,
        y = ts.loc[anomaly_true.index],
        mode = 'markers',  name = anomaly_true.name,marker= {'color':'yellow','size':10,'symbol':'star-open','line_width':2})
        fig.add_trace(status)   
    fig.show()
    
def plot_anomaly_window(ts,anomaly_pred = None,file_name = 'file',window='1h'):
    fig = go.Figure()
    yhat = go.Scatter(
    x = ts.index,
    y = ts,
    mode = 'lines',  name = ts.name)
    fig.add_trace(yhat)
    if  anomaly_pred is not None:
        for i in anomaly_pred.index:
            fig.add_vrect(x0=i - pd.Timedelta(window),x1= i,line_width=0, fillcolor="red", opacity=0.2) 
    fig.show()

In [ ]:
import pandas as pd 
df = pd.read_csv(r"附件2-行业日负荷数据.csv")
metrics_df=pd.pivot_table(df,values='有功功率最大值（kw）',index='数据时间',columns='行业类型')
metrics_df.reset_index(inplace=True)
metrics_df.fillna(0,inplace=True)
metrics_df.head()
metrics_df = metrics_df.rename(columns={'数据时间':'Time'})

In [ ]:
df = metrics_df
df['Time']= pd.to_datetime(df['Time'])
df.set_index('Time',inplace=True)
col = df.columns[0]
df.plot()

In [ ]:
col = df.columns[1]
min_t = df[col].quantile(0.05)
max_t = df[col].quantile(0.95)
df[col+'threshold'] = (df[col].clip(lower = min_t,upper=max_t) != df[col])
plot_anomaly(df[col],anomaly_pred = df[df[col+'threshold']==True][col+'threshold'],anomaly_true=None,file_name = 'file')
df[col+'突变量级'] = df[col].diff()
a = pd.DataFrame(df[df[col]>df[col].quantile(0.95)][[col,col+'突变量级']])
b = pd.DataFrame(df[df[col]<df[col].quantile(0.05)][[col,col+'突变量级']])
pd.concat([a,b]).to_csv('大工业用电突变.csv',index = 1)

In [ ]:
col = df.columns[2]
print(col)
min_t = df[col].quantile(0.05)
max_t = df[col].quantile(0.95)
df[col+'threshold'] = (df[col].clip(lower = min_t,upper=max_t) != df[col])
plot_anomaly(df[col],anomaly_pred = df[df[col+'threshold']==True][col+'threshold'],anomaly_true=None,file_name = 'file')
df[col+'突变量级'] = df[col].diff()
a = pd.DataFrame(df[df[col]>df[col].quantile(0.95)][[col,col+'突变量级']])
b = pd.DataFrame(df[df[col]<df[col].quantile(0.05)][[col,col+'突变量级']])
pd.concat([a,b]).to_csv(col+'突变.csv',index=1)

In [ ]:
col = df.columns[3]
print(col)
min_t = df[col].quantile(0.05)
max_t = df[col].quantile(0.95)
df[col+'threshold'] = (df[col].clip(lower = min_t,upper=max_t) != df[col])
plot_anomaly(df[col],anomaly_pred = df[df[col+'threshold']==True][col+'threshold'],anomaly_true=None,file_name = 'file')
df[col+'突变量级'] = df[col].diff()
a = pd.DataFrame(df[df[col]>df[col].quantile(0.95)][[col,col+'突变量级']])
b = pd.DataFrame(df[df[col]<df[col].quantile(0.05)][[col,col+'突变量级']])
pd.concat([a,b]).to_csv(col+'突变.csv',index=1)

In [ ]:
col = df.columns[0]
print(col)
min_t = df[col].quantile(0.05)
max_t = df[col].quantile(0.95)
df[col+'threshold'] = (df[col].clip(lower = min_t,upper=max_t) != df[col])
plot_anomaly(df[col],anomaly_pred = df[df[col+'threshold']==True][col+'threshold'],anomaly_true=None,file_name = 'file')
df[col+'突变量级'] = df[col].diff()
a = pd.DataFrame(df[df[col]>df[col].quantile(0.95)][[col,col+'突变量级']])
b = pd.DataFrame(df[df[col]<df[col].quantile(0.05)][[col,col+'突变量级']])
pd.concat([a,b]).to_csv(col+'突变.csv',index=1)